In [7]:
spark_session.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7fded256ca60>>

In [1]:
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, FloatType
from pyspark.sql.functions import col

import h5py
import numpy as np
import pandas as pd
import glob

In [2]:
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.130:7077") \
        .appName("John2")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .config("spark.hadoop.fs.defaultFS", "hdfs://192.168.2.130:9000")\
        .getOrCreate()

sc = spark_session.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 15:19:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
columns = StructType([
        StructField('danceability', DoubleType(), nullable=True),
        StructField('song_hotttnesss', DoubleType(), nullable=True),
        StructField('energy', DoubleType(), nullable=True),
        StructField('duration', DoubleType(), nullable=True),
        StructField('key', IntegerType(), nullable=True),
        StructField('loudness', DoubleType(), nullable=True),
        StructField('tempo', DoubleType(), nullable=True),
        StructField('time_signature', DoubleType(), nullable=True),
        StructField('year', IntegerType(), nullable=True)
    ])

def init_df():
    df = spark_session.createDataFrame([], columns)
    return df

In [4]:
import io

def add_song(df, path):
    binary = spark_session.read.format("binaryFile").load(path)
    content = io.BytesIO(binary.first()['content'])
    with h5py.File(content, 'r') as file:
        new_song_values = Row(
            loudness = float(file['analysis']['songs'][0][23]),  # DoubleType
            year = int(file['musicbrainz']['songs'][0][1])  # IntegerType
        )
        new_song = spark_session.createDataFrame([new_song_values], columns)
    return df.union(new_song)

In [5]:
df = init_df()

def list_h5_files(path):
    hadoop_conf = sc._jsc.hadoopConfiguration()
    fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(hadoop_conf)
    files = []
    status = fs.listStatus(sc._jvm.org.apache.hadoop.fs.Path(path))
    for file_status in status:
        file_path = file_status.getPath()
        if file_status.isDirectory():
            files.extend(list_h5_files(file_path.toString()))
        elif file_path.toString().endswith(".h5"):
            files.append(file_path.toString())
    return files

base_directory = "hdfs://192.168.2.130:9000/user/MillionSongSubset"
song_paths = list_h5_files(base_directory)

for i in range(100):  # Max 10000
    if i % 10 == 0:
        print(i)
    df = add_song(df, song_paths[i])

path = "hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5"

df.show()
df.printSchema()
print(df.count())
print(df.rdd.getNumPartitions())

0


PySparkValueError: [LENGTH_SHOULD_BE_THE_SAME] obj and fields should be of the same length, got 2 and 9.

In [ ]:
df2 = init_df()
song_data_rdd = spark_session.sparkContext.parallelize(song_paths[:10]).map((lambda x: add_song(df2, x)))

In [ ]:
spark_session.stop